In [ ]:
import napari
import numpy as np
from aicsimageio import AICSImage

import pandas as pd
from skimage import measure


import tifffile as tf
import matplotlib.pyplot as plt
import xarray as xr
from tifffile import tifffile
import tifftools
import os
from matplotlib import pyplot as plt
from os.path import sep
from skimage import io
from PIL import Image
import imageio


from skimage.measure import regionprops_table

import seaborn as sns

import czifile
from czifile import CziFile

import math
import matplotlib.colors as mcolors

In [2]:
fly3 = '/Users/sansachen/Downloads/lab/confocal/7_zipursky/K/03222025/SC-0322-fly3.czi'

In [3]:
# import image:
img = AICSImage(fly3)

# Get the data in (C, Z, Y, X)
# C: Channels (e.g., the different lasers/fluorophores); Z: Z-stacks (slices in depth); Y and X: The spatial dimensions (height and width)
# i don't have S or T.
data = img.get_image_data("CZYX", S=0, T=0)

# checked using "previous TQ method": 568 is first channel, 488 is second channel.
red_channel   = data[0]
green_channel = data[1]

In [ ]:
# open image in napari to draw labels:
viewer = napari.Viewer()

# Add the red channel
viewer.add_image(
    red_channel, 
    name='568 Channel',
    colormap='red',
    #contrast_limits=(100, 2500), # adjust this if needed
    blending='additive'
)
# Add the green channel
viewer.add_image(
    green_channel, 
    name='488 Channel',
    colormap='green',
    #contrast_limits=(50, 800), # adjust this if needed
    blending='additive'
)

napari.run()

In [6]:
# File paths
label_file = '/Users/sansachen/Downloads/lab/confocal/7_zipursky/K/03222025/tdt.tif'


# Load the label mask (the TIFF file you saved from napari)
label_mask = tf.imread(label_file)
# for the label tif file obtained in previous TQ method, need the below precessing:
#squeezed_mask = np.squeeze(label_mask) # so change from shape: (1, 2, Z, Y, X) to (2, Z, Y, X)
#squeezed_mask_green = squeezed_mask[1] # so only extract the green (second) channel.
print("Label mask shape:", label_mask.shape)
print("Green channel shape:", green_channel.shape)

# Check that the dimensions match (e.g., both are 3D)
if green_channel.shape != label_mask.shape:
    raise ValueError("The dimensions of the green channel and the label mask do not match. "
                     "Please verify that your label mask corresponds to the correct z, y, and x dimensions.")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sansachen/Downloads/lab/confocal/7_zipursky/K/03222025/tdt.tif'

In [ ]:
props_green = regionprops_table(
    label_mask, 
    intensity_image=green_channel,
    properties=('label', 'area', 'mean_intensity', 'max_intensity', 'min_intensity')
)
df_green = pd.DataFrame(props_green)
# Rename columns to indicate green measurements
df_green = df_green.rename(columns={
    'label': 'label',
    'area': 'area',
    'mean_intensity': 'green_mean_intensity',
    'max_intensity': 'green_max_intensity',
    'min_intensity': 'green_min_intensity'
})

props_red = regionprops_table(
    label_mask, 
    intensity_image=red_channel,
    properties=('mean_intensity', 'max_intensity', 'min_intensity')
)
df_red = pd.DataFrame(props_red)
# Rename columns to indicate red measurements
df_red = df_red.rename(columns={
    'mean_intensity': 'red_mean_intensity',
    'max_intensity': 'red_max_intensity',
    'min_intensity': 'red_min_intensity'
})

df_combined = pd.concat([df_green, df_red], axis=1)
df_combined